## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Riviere-Au-Renard,CA,48.9957,-64.3971,50.29,96,100,3.58,overcast clouds
1,1,Vostok,RU,46.4856,135.8833,46.76,64,33,2.64,scattered clouds
2,2,Zhangye,CN,38.9342,100.4517,53.31,14,100,5.64,overcast clouds
3,3,San Javier,ES,37.8063,-0.8374,71.64,79,21,3.44,few clouds
4,4,Kapaa,US,22.0752,-159.3190,82.38,80,40,9.22,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
671,671,Oktyabrskiy,RU,54.4815,53.4710,45.03,91,84,5.70,broken clouds
672,672,Mocuba,MZ,-16.8392,36.9856,65.50,97,63,3.09,light rain
673,673,Anadyr,RU,64.7500,177.4833,46.90,57,0,4.47,clear sky
674,674,Sao Miguel Do Araguaia,BR,-13.2750,-50.1628,81.09,57,87,5.48,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature, in Farenheit, you would like for your trip? "))
max_temp = float(input("What is the maximum temperature, in Farenheit, you would like for your trip? "))

What is the minimum temperature, in Farenheit, you would like for your trip? 50
What is the maximum temperature, in Farenheit, you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Riviere-Au-Renard,CA,48.9957,-64.3971,50.29,96,100,3.58,overcast clouds
2,2,Zhangye,CN,38.9342,100.4517,53.31,14,100,5.64,overcast clouds
3,3,San Javier,ES,37.8063,-0.8374,71.64,79,21,3.44,few clouds
5,5,Bluff,NZ,-46.6000,168.3333,52.81,89,100,7.76,light rain
7,7,Turayf,SA,31.6725,38.6637,68.25,49,0,6.91,clear sky
...,...,...,...,...,...,...,...,...,...,...
668,668,Strelka,RU,58.0728,93.0344,56.21,68,53,2.62,broken clouds
669,669,Noumea,NC,-22.2763,166.4572,78.01,99,53,8.10,broken clouds
670,670,Sarkand,KZ,45.4100,79.9186,56.35,52,38,5.53,scattered clouds
672,672,Mocuba,MZ,-16.8392,36.9856,65.50,97,63,3.09,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.count()

City_ID                395
City                   395
Country                395
Lat                    395
Lng                    395
Max Temp               395
Humidity               395
Cloudiness             395
Wind Speed             395
Current Description    395
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Riviere-Au-Renard,CA,50.29,overcast clouds,48.9957,-64.3971,
2,Zhangye,CN,53.31,overcast clouds,38.9342,100.4517,
3,San Javier,ES,71.64,few clouds,37.8063,-0.8374,
5,Bluff,NZ,52.81,light rain,-46.6000,168.3333,
7,Turayf,SA,68.25,clear sky,31.6725,38.6637,
...,...,...,...,...,...,...,...
668,Strelka,RU,56.21,broken clouds,58.0728,93.0344,
669,Noumea,NC,78.01,broken clouds,-22.2763,166.4572,
670,Sarkand,KZ,56.35,scattered clouds,45.4100,79.9186,
672,Mocuba,MZ,65.50,light rain,-16.8392,36.9856,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city. 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        hotel_df.loc[index,"Hotel Name"] = np.nan
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
hotel_df.isnull().sum()

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.count()

In [ ]:
clean_hotel_df

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 30.0), zoom_level=3)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig